# Planeación de rutas para la exploración de Marte

---

- Diseño de agentes inteligentes
- TC2032.101
- Profesor: Juan Emmanuel Martínez Ledesma

Equipo 7:

| Nombre | Matrícula |
| ----- | ------ |
| Juan Pablo Echeagaray González | A00830646 |
| Emily Rebeca Méndez Cruz | A00830768 |
| Erika Martínez Meneses | A01028621 |
| Oscar Antonio Banderas Álvarez | A01568492 |
| César Guillermo Vázquez Alvarez | A01197857 |

- 8 de marzo del 2022


## Instrucciones

Una fuente de información importante para la exploración de Marte son las imágenes obtenidas con orbitadores que están continuamente enviando información sobre la superficie de nuestro planeta vecino. En la página de [HiRISE](https://www.uahirise.org/dtm/) se encuentra disponible una gran cantidad de imágenes en alta resolución captadas a través de diversos satélites con instrumentos capaces de medir la profundidad y otras características del suelo marciano. Estos datos ayudaron a determinar el lugar donde aterrizó el Rover Perseverance en la última misión llevada a cabo por parte de la NASA en el planeta rojo.

Las siguientes figuras son representaciones en 3D de una pequeña parte de la superficie de Marte. Los ejes representan distancias en metros, y los colores indican las partes más profundas (en rojo) y más elevadas (en amarillo claro). Se toma como nivel 0 la parte más profundad de la superficie mostrada. Estos datos se encuentran en el sitio de [HiRISE](https://www.uahirise.org/dtm/).

![](img\2022-03-07-11-44-05.png)

![](img\2022-03-07-11-45-21.png)

![](img\2022-03-07-11-45-35.png)

La información mostrada con anterioridad se puede representar en una imagen como la que se muestra a continuación. El color de cada pixel de la imagen representa la altura en dicha posición. Las secciones en gris representan áreas donde no hay información satelital.

---

![](img\2022-03-07-11-46-01.png)

Los datos en el sitio [HiRISE](https://www.uahirise.org/dtm/) están disponibles en archivos IMG con información topográfica. En los archivos originales, la superficie anterior tiene dimensiones de 7557 pixeles de ancho y 18143 pixeles de alto. Cada pixel representa un área de 1.0017 metros x 1.0017 metros. Estos datos no son colores RGB, son valores de altura, y la imagen mostrada se conoce como imagen de falso color, ya que se construye artificialmente asignando un color a cada pixel de acuerdo a alguna dimensión o medida (en este caso, valores de profundidad o altura). A estas imágenes se les llaman mapas de altura. Los valores en gris tienen un valor de altura de -1, y representan áreas donde no hay información válida.

El archivo de datos de la superficie mostrada se puede descargar [aquí](https://experiencia21.tec.mx/courses/241470/files/78461927?wrap=1) . Por otro lado, el código que carga el archivo IMG y genera las imágenes anteriores se puede descargar [aquí](https://experiencia21.tec.mx/courses/241470/files/78461984?wrap=1). Debido a que el tamaño del archivo de datos es de alrededor de un 1Gb, la imagen original se escala a un tamaño menor. Con ello, las imágenes mostradas tienen dimensiones de 756 x 1814, y cada pixel representa a un área de 10.0174 x 10.0174.

Debido a que los datos se manejan como una imagen, el sistema de coordenadas de la matriz  de datos está volteado en el eje vertical. En una imagen, la parte superior está representada en el primer renglón de la matriz de datos. Por ello, para calcular el renglón $r$ y columna $c$ de una coordenada $x, y$, realice lo siguiente:

$$\begin{gather*}
r = n_r - \text{round}\left(\frac{y}{scale}\right) \\
c = \text{round}\left(\frac{y}{scale}\right)
\end{gather*}$$

Donde $n_r$ y $n_c$ son los números de renglones y columnas de la imagen o matriz de datos (en este caso 1814 y 756), y la escala indica la cantidad de metros por cada pixel (10.0174).

En esta actividad, probarás algoritmos de búsqueda para la planeación de rutas en el mapa de altura mostrado en esta actividad. Para ello, se pide que realicen las siguientes tareas.

### Prueba de algoritmos de búsqueda

Para el mapa de alturas presentado en esta actividad, utilicen al menos cuatro algoritmos de búsqueda para determinar una posible ruta de navegación desde la posición $x = 2850, y = 6400$ hasta el punto $x = 3150, y = 6800$. Entre los algoritmos de búsqueda, debe incluir el método A*. 

Para determinar rutas posibles, consideren lo siguiente:
- El Rover explorador puede moverse de un pixel a otro contiguo en el mapa en cualquier dirección, pero sólo si la diferencia de alturas entre la posición actual y la siguiente posición es menor a 0.25 metros (el programa debe manejar este valor como parámetro que pueda ser modificado fácilmente).
- Los pixeles del mapa con una altura de -1 no son válidos y el Rover no se puede mover hacia ellos.

Para este punto, es necesario indicar la distancia recorrida por el Rover. Además, contesten las siguientes preguntas:
- ¿Qué algoritmos lograron encontrar una ruta válida? 
- ¿Es necesario utilizar búsquedas informadas para este caso?
- ¿Qué función heurística resultó adecuada para este problema?


### Rendimiento de los algoritmos de búsqueda para rutas cortas y largas

Seleccionen uno de los algoritmos de búsqueda probados en el apartado anterior de esta fase del proyecto, y determinen con éste rutas de navegación para al menos dos parejas de coordenadas que seleccionen ustedes mismos, de tal manera que el punto de partida y el objetivo no estén a más de $500$ metros. Hagan lo mismo, pero con al menos dos parejas que tengan una distancia entre inicio y fin de mas de $1000$ metros y menos de $5000$ metros. Finalmente, prueben parejas de coordenadas que estén a una distancia mayor a $10000$ metros.

Contesten las siguientes preguntas:
- ¿En qué casos el algoritmo es capaz de resolver el problema en un tiempo aceptable? 
- En los casos que el algoritmo no encuentra un resultado, ¿qué acciones se podrían realizar para ayudar al algoritmo a resolver el problema?


## Implementación

In [1]:
# Basic dependencies
from simpleai.search import SearchProblem
from simpleai.search import breadth_first, uniform_cost, depth_first, astar
import numpy as np

from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Cascadia Code; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")


### Características del problema

In [2]:
SCALE = 10.0174
mars_map = np.load('mars_map.npy')
NR, NC = mars_map.shape

# Start and goal positions
x0, y0 = 2850, 6400
xf, yf = 3150, 6800
MAX_DIFF = 0.25
UNTRAVERSABLE = -1
STEP_SIZE = 1


### Funciones auxiliares

In [3]:
def pos_to_index(pos: list[int]) -> list[int]:
    nr = NR - round(pos[0] / SCALE)
    nc = round(pos[1] / SCALE)
    return [nr, nc]


def coord_to_state(coord: list[int]) -> str:
    return f'{coord[0]},{coord[1]}'


def state_to_coord(state: str) -> list[int]:
    return tuple([int(x) for x in state.split(',')])


def report(result):
    print(f'Found path with length {len(result.path())}')
    print(f'Path: {result.path()}')
    print(f'Cost: {result.cost}')


def tester(search_method, imax, delta=20):

    print(f'Testing {search_method.__name__}')
    for i in np.linspace(imax, 0, delta):
        problem = mars_route_finding(START_STATE, GOAL_STATE, mars_map, i * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)
        result = search_method(problem, graph_search=True)
        report(result)
        print(f'Max difference (factor {i}): {i * MAX_DIFF}')


def distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)


### Estados iniciales y finales

In [4]:
# Coordenadas iniciales
START = pos_to_index((x0, y0))
GOAL = pos_to_index((xf, yf))

# Conversión a estado
START_STATE = coord_to_state(START)
GOAL_STATE = coord_to_state(GOAL)


### Implementación del problema

In [5]:
class mars_route_finding(SearchProblem):

    def __init__(self, initial_state: str, goal_state: str, mars_map: np.array, max_height: float, not_traversable: int, step_size: int):

        self.mars_map = mars_map
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.max_height = max_height
        self.not_traversable = not_traversable
        self.step_size = step_size

        print("Initial state: ", self.initial_state)
        print("Goal state: ", self.goal_state)

        SearchProblem.__init__(self, initial_state)

    def actions(self, state):
        mars_map = self.mars_map
        actions = []

        pos_directions = ['U', 'D', 'L', 'R', 'UL', 'UR', 'DL', 'DR']
        x0, y0 = state_to_coord(state)

        for action in pos_directions:
            x, y = state_to_coord(self.result(state, action))
            height_difference = abs(mars_map[x0, y0] - mars_map[x, y])

            if (mars_map[x, y] != self.not_traversable):
                if height_difference <= self.max_height:
                    # print(f'[{x}, {y}], action: {action}, height_difference: {height_difference}')
                    actions.append(action)
    
        return actions

    def result(self, state, action):
        x, y = state_to_coord(state)
        step_size = self.step_size

        if action.count('U'):
            x -= step_size
        elif action.count('D'):
            x += step_size
        elif action.count('L'):
            y -= step_size
        elif action.count('R'):
            y += step_size
        elif action.count('UL'):
            x -= step_size
            y -= step_size
        elif action.count('UR'):
            x -= step_size
            y += step_size
        elif action.count('DL'):
            x += step_size
            y -= step_size
        elif action.count('DR'):
            x += step_size
            y += step_size
        else:
            print('No se encontró la acción')
        
        new_state = coord_to_state([x, y])
        return new_state

    def is_goal(self, state):

        return state == self.goal_state

    def cost(self, state, action, state2):
        alpha, beta = 1, 1

        x0, y0 = state_to_coord(state)
        xn, yn = state_to_coord(state2)

        distance = np.sqrt((x0 - xn)**2 + (y0 - yn)**2)
        height_difference = abs(self.mars_map[x0, y0] - self.mars_map[xn, yn])

        return alpha * distance + beta * height_difference

    def heuristic(self, state):
        x, y = state_to_coord(state)
        xg, yg = state_to_coord(self.goal_state)
        distance = np.sqrt((x - xg)**2 + (y - yg)**2)

        return distance


### Prueba de algoritmos de búsqueda

In [6]:
my_problem = mars_route_finding(START_STATE, GOAL_STATE, mars_map, 5.5 * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)


Initial state:  1530,639
Goal state:  1501,679


#### Breadth-first search

In [7]:
%%timeit
breadth_result = breadth_first(my_problem, graph_search=True)

783 ms ± 79.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
breadth_result = breadth_first(my_problem, graph_search=True)

In [9]:
report(breadth_result)

Found path with length 70
Path: [(None, '1530,639'), ('U', '1529,639'), ('U', '1528,639'), ('U', '1527,639'), ('U', '1526,639'), ('U', '1525,639'), ('U', '1524,639'), ('U', '1523,639'), ('U', '1522,639'), ('R', '1522,640'), ('R', '1522,641'), ('R', '1522,642'), ('R', '1522,643'), ('R', '1522,644'), ('U', '1521,644'), ('U', '1520,644'), ('R', '1520,645'), ('R', '1520,646'), ('R', '1520,647'), ('R', '1520,648'), ('U', '1519,648'), ('R', '1519,649'), ('U', '1518,649'), ('R', '1518,650'), ('R', '1518,651'), ('R', '1518,652'), ('R', '1518,653'), ('R', '1518,654'), ('R', '1518,655'), ('R', '1518,656'), ('U', '1517,656'), ('U', '1516,656'), ('U', '1515,656'), ('U', '1514,656'), ('U', '1513,656'), ('U', '1512,656'), ('U', '1511,656'), ('U', '1510,656'), ('U', '1509,656'), ('U', '1508,656'), ('U', '1507,656'), ('U', '1506,656'), ('U', '1505,656'), ('U', '1504,656'), ('U', '1503,656'), ('U', '1502,656'), ('U', '1501,656'), ('R', '1501,657'), ('R', '1501,658'), ('R', '1501,659'), ('R', '1501,660'

#### Depth-first search

In [10]:
depth_result = depth_first(my_problem, graph_search=True)

KeyboardInterrupt: 

In [11]:
report(depth_result)

NameError: name 'depth_result' is not defined

#### Uniform cost search

In [13]:
%%timeit
uniform_result = uniform_cost(my_problem, graph_search=True)

1.11 s ± 86.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
uniform_result = uniform_cost(my_problem, graph_search=True)

In [15]:
report(uniform_result)

Found path with length 70
Path: [(None, '1530,639'), ('R', '1530,640'), ('R', '1530,641'), ('R', '1530,642'), ('R', '1530,643'), ('R', '1530,644'), ('U', '1529,644'), ('R', '1529,645'), ('U', '1528,645'), ('U', '1527,645'), ('U', '1526,645'), ('U', '1525,645'), ('U', '1524,645'), ('U', '1523,645'), ('R', '1523,646'), ('R', '1523,647'), ('U', '1522,647'), ('R', '1522,648'), ('R', '1522,649'), ('U', '1521,649'), ('U', '1520,649'), ('R', '1520,650'), ('R', '1520,651'), ('R', '1520,652'), ('R', '1520,653'), ('R', '1520,654'), ('R', '1520,655'), ('R', '1520,656'), ('U', '1519,656'), ('U', '1518,656'), ('R', '1518,657'), ('U', '1517,657'), ('U', '1516,657'), ('R', '1516,658'), ('U', '1515,658'), ('U', '1514,658'), ('U', '1513,658'), ('U', '1512,658'), ('U', '1511,658'), ('R', '1511,659'), ('R', '1511,660'), ('U', '1510,660'), ('U', '1509,660'), ('U', '1508,660'), ('R', '1508,661'), ('U', '1507,661'), ('R', '1507,662'), ('U', '1506,662'), ('R', '1506,663'), ('U', '1505,663'), ('R', '1505,664'

#### A* search

In [16]:
%%timeit
astar_result = astar(my_problem, graph_search=True)

462 ms ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
astar_result = astar(my_problem, graph_search=True)

In [18]:
report(astar_result)

Found path with length 70
Path: [(None, '1530,639'), ('R', '1530,640'), ('R', '1530,641'), ('R', '1530,642'), ('R', '1530,643'), ('R', '1530,644'), ('U', '1529,644'), ('R', '1529,645'), ('U', '1528,645'), ('U', '1527,645'), ('U', '1526,645'), ('U', '1525,645'), ('U', '1524,645'), ('U', '1523,645'), ('R', '1523,646'), ('R', '1523,647'), ('U', '1522,647'), ('R', '1522,648'), ('R', '1522,649'), ('U', '1521,649'), ('U', '1520,649'), ('R', '1520,650'), ('R', '1520,651'), ('R', '1520,652'), ('R', '1520,653'), ('R', '1520,654'), ('R', '1520,655'), ('R', '1520,656'), ('U', '1519,656'), ('U', '1518,656'), ('R', '1518,657'), ('U', '1517,657'), ('U', '1516,657'), ('R', '1516,658'), ('U', '1515,658'), ('U', '1514,658'), ('U', '1513,658'), ('U', '1512,658'), ('U', '1511,658'), ('R', '1511,659'), ('R', '1511,660'), ('R', '1511,661'), ('R', '1511,662'), ('U', '1510,662'), ('R', '1510,663'), ('R', '1510,664'), ('U', '1509,664'), ('U', '1508,664'), ('U', '1507,664'), ('R', '1507,665'), ('R', '1507,666'

- 3 de los 4 algoritmos de búsqueda fueron capaces de encontrar rutas válidas, el único que no pudo hacerlo fue el de `depth_first_search`, el número de nodos por visitar es simplemente enorme (y como en este caso la meta no estaba tan lejos).
- 2 de los algoritmos no informados encontraron una ruta válida en tiempos aceptables, pero el algoritmo `A*` fue el más veloz de todos, además que lo logró con el camino con el menor costo
- Como función heurística propusimos la distancia euclideana entre el punto actual y la meta.

### Rendimiento de los algoritmos de búsqueda

#### Menos de 500 metros de separación

In [19]:
# ~ 500 metros
x0, y0 = 2850, 6400
xf, yf = 2400, 6500
GOAL_500 = pos_to_index((xf, yf))
GOAL_STATE_500 = coord_to_state(GOAL_500)
print(f'Distancia entre los puntos = {distance([x0, y0], [xf, yf])}')

Distancia entre los puntos = 460.9772228646444


In [20]:
my_problem_500 = mars_route_finding(START_STATE, GOAL_STATE_500, mars_map, 5.5 * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)

Initial state:  1530,639
Goal state:  1575,649


In [21]:
result_500 = astar(my_problem, graph_search=True)

In [22]:
report(result_500)

Found path with length 70
Path: [(None, '1530,639'), ('R', '1530,640'), ('R', '1530,641'), ('R', '1530,642'), ('R', '1530,643'), ('R', '1530,644'), ('U', '1529,644'), ('R', '1529,645'), ('U', '1528,645'), ('U', '1527,645'), ('U', '1526,645'), ('U', '1525,645'), ('U', '1524,645'), ('U', '1523,645'), ('R', '1523,646'), ('R', '1523,647'), ('U', '1522,647'), ('R', '1522,648'), ('R', '1522,649'), ('U', '1521,649'), ('U', '1520,649'), ('R', '1520,650'), ('R', '1520,651'), ('R', '1520,652'), ('R', '1520,653'), ('R', '1520,654'), ('R', '1520,655'), ('R', '1520,656'), ('U', '1519,656'), ('U', '1518,656'), ('R', '1518,657'), ('U', '1517,657'), ('U', '1516,657'), ('R', '1516,658'), ('U', '1515,658'), ('U', '1514,658'), ('U', '1513,658'), ('U', '1512,658'), ('U', '1511,658'), ('R', '1511,659'), ('R', '1511,660'), ('R', '1511,661'), ('R', '1511,662'), ('U', '1510,662'), ('R', '1510,663'), ('R', '1510,664'), ('U', '1509,664'), ('U', '1508,664'), ('U', '1507,664'), ('R', '1507,665'), ('R', '1507,666'

#### 1000 a 5000 metros de separación

In [23]:
# ~ 1000 metros
xf, yf =  1050, 7412
distance_1000 = distance([x0, y0], [xf, yf])
GOAL_1000 = pos_to_index((xf, yf))
GOAL_STATE_1000 = coord_to_state(GOAL_1000)
print(f'Distancia entre los puntos = {distance_1000}')


Distancia entre los puntos = 2064.9803873160636


In [24]:
mars_map[tuple(GOAL_1000)]

172.23093505859387

In [25]:
my_problem_1000 = mars_route_finding(START_STATE, GOAL_STATE_1000, mars_map, 5.5 * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)


Initial state:  1530,639
Goal state:  1710,740


In [26]:
result_1000 = astar(my_problem_1000, graph_search=True)


In [27]:
report(result_1000)

Found path with length 302
Path: [(None, '1530,639'), ('R', '1530,640'), ('R', '1530,641'), ('R', '1530,642'), ('R', '1530,643'), ('R', '1530,644'), ('U', '1529,644'), ('R', '1529,645'), ('U', '1528,645'), ('U', '1527,645'), ('U', '1526,645'), ('U', '1525,645'), ('U', '1524,645'), ('U', '1523,645'), ('R', '1523,646'), ('R', '1523,647'), ('U', '1522,647'), ('R', '1522,648'), ('R', '1522,649'), ('U', '1521,649'), ('U', '1520,649'), ('R', '1520,650'), ('R', '1520,651'), ('R', '1520,652'), ('R', '1520,653'), ('R', '1520,654'), ('R', '1520,655'), ('D', '1521,655'), ('R', '1521,656'), ('R', '1521,657'), ('R', '1521,658'), ('R', '1521,659'), ('R', '1521,660'), ('R', '1521,661'), ('D', '1522,661'), ('D', '1523,661'), ('D', '1524,661'), ('D', '1525,661'), ('D', '1526,661'), ('D', '1527,661'), ('R', '1527,662'), ('D', '1528,662'), ('D', '1529,662'), ('D', '1530,662'), ('D', '1531,662'), ('D', '1532,662'), ('R', '1532,663'), ('R', '1532,664'), ('D', '1533,664'), ('R', '1533,665'), ('D', '1534,665

#### Mayor a 10,000 metros de separación

In [160]:
x0, y0 = 2850, 6400
xf, yf = 1150, 2480

# 2850, 6400
# 1150, 7432

# cannot find a path
# x0, y0 = 2850, 6400
# xf, yf = 1150, 2480
START_10000 = pos_to_index((x0, y0))
START_STATE_10000 = coord_to_state(START_10000)
GOAL_10000 = pos_to_index((xf, yf))
GOAL_STATE_10000 = coord_to_state(GOAL_10000)
step = 6

print(f'Distancia entre los puntos = {distance([x0, y0], [xf, yf])}')


Distancia entre los puntos = 4272.750870341027


In [161]:
my_problem_10000 = mars_route_finding(START_STATE_10000, GOAL_STATE_10000, mars_map, 5.5 * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)


Initial state:  1530,639
Goal state:  1700,248


In [162]:
result_10000 = astar(my_problem_10000, graph_search=True)


In [163]:
report(result_10000)

Found path with length 634
Path: [(None, '1530,639'), ('R', '1530,640'), ('R', '1530,641'), ('R', '1530,642'), ('R', '1530,643'), ('R', '1530,644'), ('U', '1529,644'), ('U', '1528,644'), ('R', '1528,645'), ('U', '1527,645'), ('U', '1526,645'), ('U', '1525,645'), ('U', '1524,645'), ('U', '1523,645'), ('R', '1523,646'), ('U', '1522,646'), ('R', '1522,647'), ('R', '1522,648'), ('U', '1521,648'), ('R', '1521,649'), ('U', '1520,649'), ('R', '1520,650'), ('R', '1520,651'), ('R', '1520,652'), ('R', '1520,653'), ('R', '1520,654'), ('R', '1520,655'), ('D', '1521,655'), ('R', '1521,656'), ('R', '1521,657'), ('R', '1521,658'), ('R', '1521,659'), ('R', '1521,660'), ('R', '1521,661'), ('D', '1522,661'), ('D', '1523,661'), ('D', '1524,661'), ('D', '1525,661'), ('D', '1526,661'), ('D', '1527,661'), ('D', '1528,661'), ('D', '1529,661'), ('D', '1530,661'), ('D', '1531,661'), ('L', '1531,660'), ('D', '1532,660'), ('D', '1533,660'), ('D', '1534,660'), ('D', '1535,660'), ('D', '1536,660'), ('D', '1537,660

Path found!

## Contribuciones

### Juan Pablo Echeagaray González

- Control de versiones del cuaderno
- Implementación de la clase del problema
- Prueba de algoritmos en diferentes escenarios

## Conclusiones

### Juan Pablo Echeagaray González